In [1]:
import os
import json
from tqdm import tqdm

os.chdir('dataset/Diff_Quality_Estimation')
msg_file = f"comments.jsonl"


In [3]:
LEN=66459
filenames = [f'cls-train-chunk-{i}' for i in range(4)]


In [11]:
with open(f"cls-train-chunk-1.jsonl", "r") as fr:
    x = [json.loads(next(fr)) for _ in range(10)]

In [8]:
x.keys()

dict_keys(['patch', 'y', 'oldf', 'idx', 'id', 'msg', 'proj', 'lang'])

In [13]:
print(x[3]['patch'])

@@ -2774,11 +2774,10 @@ class IrGraph(object):
 
 class Program(object):
     """
-    Python Program. Beneath it is a ProgramDesc, which is used for
-    create c++ Program. A program is a self-contained programing
-    language like container. It has at least one Block, when the
-    control flow op like conditional_block, while_op is included,
+    Create Python Program.  It has at least one :ref:`api_guide_Variable_en`, when the
+    control flow op like conditional_block, while :ref:`api_fluid_layers_While` is included,
     it will contain nested block.
+
     Please reference the framework.proto for details.
 
     A set of Program usually contains startup program and main program.



# create comments.jsonl for comments

collect all the PR comments into a single file for further analysis

In [118]:

with open(msg_file, "w") as fw:
    for filename in filenames:
        with open(f"{filename}.jsonl", "r") as fr:
            i, j = 0, 0
            for line in tqdm(fr):
                data = json.loads(line)
                i += 1
                if 'msg' in data and len(data['msg']):
                    j += 1
                    fw.write(json.dumps(data['msg'])+'\n')
            print(f"{j}/{i} are saved to {msg_file}")


66459it [00:07, 9116.52it/s]


33166/66459 are saved to comments.jsonl


66459it [00:07, 8984.82it/s]


33233/66459 are saved to comments.jsonl


66459it [00:07, 8801.04it/s]


33272/66459 are saved to comments.jsonl


66459it [00:07, 9048.26it/s]

33247/66459 are saved to comments.jsonl


In [10]:
from smells import smells

In [15]:
head_re = r"^@@ -(\d+),(\d+) \+(\d+),(\d+) @@ "

# Extract codes to separate files

In [18]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from baseform import is_base_form_name
from filenames import all_names

lemmatizer = WordNetLemmatizer()
i, j = 0, {base: dict() for base in smells}
for filename in all_names:
    with open(f'{filename}.jsonl') as fr:
        with open(f'{filename}-ready.jsonl', 'w') as fw:
            for line in tqdm(fr):
                i += 1
                data = json.loads(line)
                lang = data['lang'] if 'lang' in data else 'unk'
                patch = re.sub(head_re, '', data["patch"])
                patch = re.sub(r"\n( |\+)", '\n', patch)
                code = re.sub(r"\n-(.*)", '\n\\1', patch)
                bad = ('msg' in data) and len(data['msg'])
                label = [0] * (1 + len(smells))
                if bad:
                    label[0] = 1
                    words = word_tokenize(data["msg"])
                    for smell_idx, smell in enumerate(smells):
                        if any(is_base_form_name(lemmatizer, word, smell) for word in words):
                            if not lang in j[smell]:
                                j[smell][lang] = 0
                            j[smell][lang] += 1
                            label[smell_idx+1] = 1
                fw.write(f"{json.dumps([code, label, 0 if lang == 'js' else -1 if lang == 'unk' else 1])}\n")



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
31252it [00:11, 2624.04it/s]
31252it [00:08, 3693.17it/s]
66459it [00:15, 4212.98it/s]
66459it [00:15, 4213.41it/s]
66459it [00:16, 4055.17it/s]
66459it [00:16, 4054.47it/s]


In [19]:
j

{'name': {'py': 227,
  'java': 328,
  'go': 548,
  'php': 93,
  'rb': 164,
  'js': 130,
  '.cs': 125,
  'c': 68,
  'cpp': 90,
  'unk': 6006},
 'remove': {'.cs': 129,
  'rb': 172,
  'go': 421,
  'cpp': 173,
  'java': 288,
  'js': 216,
  'py': 249,
  'php': 97,
  'c': 83,
  'unk': 8265},
 'work': {'php': 47,
  'js': 128,
  'go': 195,
  'java': 106,
  'py': 128,
  '.cs': 69,
  'cpp': 67,
  'rb': 120,
  'c': 42,
  'unk': 3705}}

In [ ]:
0